In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [2]:
df = pd.read_csv('streamer_df')

In [3]:
# creates a dataframe for social media plattforms scraped from twitch for a given df with usernames in it
def get_social(df):
    ''' this function scrapes the URL to youtube and twitter for a username-list'''
    import time
    t=5
    usernames = df.username.to_list()
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    chrome_driver_binary = "/Users/frckles/drivers/chromedriver"
    dict = {}
    for user in usernames:
        URL= f'https://www.twitch.tv/{user}/about'
        driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
        driver.get(URL)
        soup=BeautifulSoup(driver.page_source)
        twitter = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "twitter.com/" in a["href"]]))
        youtube = list(set([a["href"] for a in soup.find_all("a", href=True) if "youtube.com/" in a["href"]]))
        instagram = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "instagram" in a["href"]]))
        discord = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "discord" in a["href"]]))
        if twitter:
            twitter = twitter[0]
        if instagram:
            instagram = instagram[0]
        if discord:
            discord = discord[0]
        dict[user] = [twitter, youtube, instagram, discord]
        time.sleep(t)
    df = pd.DataFrame.from_dict(dict, orient='index')
    df['Twitter'] = df[0].apply(lambda x: None if x == [] else x)
    df['YouTube'] = df[1].apply(lambda x: None if x == [] else x)
    df['Instagram'] = df[2].apply(lambda x: None if x == [] else x)
    df['Discord'] = df[3].apply(lambda x: None if x == [] else x)
    df.index.name = 'username'
    df = df.reset_index()
    return df[['username', 'Twitter', 'YouTube', 'Instagram', 'Discord']]

In [ ]:
socials = get_social(df)
df_socs = socials.copy()

In [4]:
def get_social_id(link):
    '''Extract streamer identifier at the end of their social media links'''
    if link[-1] == '/':
        link = link[:-1]
    start = link.rfind('/')
    id = link[start + 1:]

    return id

def get_yt_channel_id(links):
    '''Return the first genuine yt channel id in a list of yt links'''
    if not links:
        return None
    channel_id = None
    for link in links:
        link = link.split('?')[0]
        print(link)
        id = get_social_id(link)
        if len(id) == 24:
            channel_id = id
            break
    
    return channel_id


def parse_streamer_socials(socials_df):
    '''Return a dataframe with relevant ids per treamer and platform'''
    for (column_name, column_data) in socials_df.iteritems():
        if column_name != 'Discord':
            if column_name == 'YouTube':
                socials_df[column_name] = socials_df[column_name].apply(get_yt_channel_id)
            socials_df[column_name] = column_data.apply(lambda x: get_social_id(x) if x else None)
            
        
    return socials_df

In [ ]:
get_yt_channel_id(test_link)

In [ ]:
parse_streamer_socials(df_socs)